In [7]:
from call_llm_with_vllm import CallLLMByVllm,VllmConfig
from call_llm_with_zny import CallLLMByZny,ZnyConfig
import pandas as pd

import asyncio
import aiohttp
import json
from tqdm import tqdm
import pandas as pd
from datetime import datetime
from asyncio import Semaphore
import time
import requests

In [3]:
config = ZnyConfig(
    url = 'https://rhm-gpt4.fc.chj.cloud/gpt4o', # 智能云GPT api
    model_name = 'gpt4o', # system:转为system和user, '':输入的message都是user
    temperature = 1, # llm输出温度，zny下的gpt4基本无效，因为是全球节点，还是会有随机性
    max_retries = 5, # 调用gpt报错后最多重试 max_retries 次
    qps = 5, # 多线程 or 异步多线程下，qps，不要超过5
    max_concurrent = 10, # 异步多线程参数，一般10或者20，太大会接口超过qps
    asyncio_flag = False, # True=异步多线程，只能python调用；False=普通多线程，Jupyter或者python均可
    query_column_name = 'prompts', # llm模型输入列名
    response_column_name = 'gpt4response', # llm模型输出列名
)

call_zny = CallLLMByZny(config)

In [2]:
def get_prompts_df(df: pd.DataFrame, oneshot_prompt:str) -> pd.DataFrame:
    df['prompts'] = df.apply(lambda row: oneshot_prompt + f"""
    ---
    下面是给出的实际问题：
    Observation:
    {row['observation']}
    Question:
    {row['user-query']}
    Answer：
    """, axis=1)
    
    return df

In [6]:
folder = '/mnt/pfs-guan-ssai/nlu/data/renhuimin/eval_tools/0_data/0628_模型收益测试/output_instruct_follow/'
df = pd.read_csv(folder+'测试集_指令遵循_角色扮演.csv.v6_0607.csv')[:5]

oneshot_prompt = '根据参考资料回答问题'
# oneshot_prompt = "你是一个车载智能语音助手，能够理解人类的指令和意图，并且给出合理的、切合问题的、没有歧视、中立的、安全的回复。对于意图不清晰的问题，你需要询问用户真实意图。"
prompt_df = get_prompts_df(df, oneshot_prompt)
gpt4_df = call_zny.get_gpt4api_df(prompt_df)
gpt4_df

,task-name,user-query,Thought,API,observation,gpt4-turbo,文心一言,correct_output,id,system,model_16b_input,predict_output,full_input,full_output,prompts,gpt4response
0,指令遵循-角色扮演-康辉,"嫦娥奔月还是奔日,请以专业主持人康辉的表达风格来回复",涉及人物知识问答，查询嫦娥奔月是奔日吗和奔月是奔日吗做参考。,<|api_start|><|kvs|>APINAME=>QASearch<|kve|><|...,[['嫦娥奔月，神话故事的背后是什么？ - 知乎 发布时间：2022-09-11 19:58...,嫦娥奔月的故事，是中国古代广为流传的一段美丽传说。在这个故事中，嫦娥并不是奔向太阳，而是飞升...,嫦娥奔月，而非奔日，这是中国古代一则脍炙人口的神话传说。在这个故事中，嫦娥因种种原因，最终选...,无,0,[unused0]system\n你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一...,[unused0]system\n[unused0]system\n你是一个名字叫做理想同学...,哎呀，各位观众朋友们，大家好！今天我们要聊的可是个经典中的经典，那就是嫦娥奔月的故事。这个故...,[unused0]system\n[unused0]system\n你是一个名字叫做理想同学...,[unused0] system[unused0] system\n你是一个名字叫做理想同学...,根据参考资料回答问题\n ---\n 下面是给出的实际问题：\n Obse...,各位观众朋友，大家好。这是康辉为您带来的特别节目。\n\n今天，我们一同来到中国神话的世界，...
1,指令遵循-角色扮演-康辉,"宇宙一共有几大行星,请以专业主持人康辉的表达风格来回复","涉及人物知识查询，查询宇宙一共有几大行星,以专业主持人康辉的表达风格来回复",<|api_start|><|kvs|>APINAME=>QASearch<|kve|><|...,[['宇宙一共有多少个行星？ - 百度知道 100dubaichi 2007-03-20 ·...,亲爱的观众朋友们，大家好，我是康辉。今天，我们来聊聊一个宇宙级的大话题——宇宙中到底有多少颗...,宇宙中的行星数量是一个庞大且令人惊叹的天文数字。然而，当我们谈论“几大行星”时，这通常指的是...,无,0,[unused0]system\n你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一...,[unused0]system\n[unused0]system\n你是一个名字叫做理想同学...,尊敬的观众朋友们，今天我们要探讨的是一个浩渺而又深邃的话题，那就是宇宙中的行星数量。在广阔无...,[unused0]system\n[unused0]system\n你是一个名字叫做理想同学...,[unused0] system[unused0] system\n你是一个名字叫做理想同学...,根据参考资料回答问题\n ---\n 下面是给出的实际问题：\n Obse...,各位观众朋友，大家好。我是康辉，欢迎收看今天的天文知识专题节目。\n\n关于宇宙中的行星数量...
2,指令遵循-角色扮演-康辉,"孔子是哪年出生的,请以专业主持人康辉的表达风格来回复",涉及人物知识查询，查询孔子的出生日期康辉的表达风格,<|api_start|><|kvs|>APINAME=>QASearch<|kve|><|...,[['孔子诞辰日 孔子诞辰日孔子孔丘（公元前551年9月28日），字仲尼，春秋时期鲁国人。孔...,尊敬的观众朋友们，大家好。今天，我们要聊一位历史上极具影响力的伟大人物，他就是我国古代著名的...,孔子，这位被后世尊称为“至圣先师”的伟大思想家和教育家，儒家学派的奠基人，其诞生之日，已铭刻...,无,0,[unused0]system\n你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一...,[unused0]system\n[unused0]system\n你是一个名字叫做理想同学...,哎呀，各位观众朋友们，今天咱们来聊一位历史上非常了不起的人物，他就是孔子，孔老夫子。孔子啊，...,[unused0]system\n[unused0]system\n你是一个名字叫做理想同学...,[unused0] system[unused0] system\n你是一个名字叫做理想同学...,根据参考资料回答问题\n ---\n 下面是给出的实际问题：\n Obse...,各位观众朋友，大家好。我是康辉。今天，我们来共同探讨一位古代思想巨匠——孔子的出生年份。\n...
3,指令遵循-角色扮演-康辉,"鱼的身体是流线型吗,请以专业主持人康辉的表达风格来回复","涉及人物知识查询，查询鱼的身体是流线型吗,请以专业主持人康辉的表达风格来回复",<|api_start|><|kvs|>APINAME=>QASearch<|kve|><|...,[['鱼类的体型大致可以归纳为下列四种基本类型： - Foodmate 发布时间：2006-...,当然，鱼的身体是流线型的。这种设计，无疑是大自然的杰作，它让鱼儿在水中游动时，能够像穿梭的箭...,鱼类的身体是否呈流线型，这一生物学特性与它们在水中的生活方式息息相关。康辉在此为您揭示答案。...,无,0,[unused0]system\n你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一...,[unused0]system\n[unused0]system\n你是一个名字叫做理想同学...,哎呀，各位观众朋友们，这个问题可真有趣。咱们今天来说一说鱼的身体，是不是流线型呢？\n其实啊...,[unused0]system\n[unused0]system\n你是一个名字叫做理想同学...,[unused0] system[unused0] system\n你是一个名字叫做理想同学...,根据参考资料回答问题\n ---\n 下面是给出的实际问题：\n Obse...,当然，许多鱼类的身体是流线型的，这是一种经过亿万年进化的适应性设计。流线型不仅使鱼类在水中游...
4,指令遵循-角色扮演-康辉,"世界上最高的山是哪个,请以专业主持人康辉的表达风格来回复","涉及人物知识问答，查询世界最高的山是哪个,请以专业主持人康辉的表达风格来回复做参考。",<|api_start|><|kvs|>APINAME=>QASearch<|kve|><|...,[['世界上最高的山是哪个？ 发布时间：2023-12-05 10:30:06；世界上最高的...,尊敬的观众朋友们，大家好，我是康辉。今天，我们要探讨一个关于自然界的奇妙话题——世界上最高的...,世界上最高的山峰，这一荣誉无疑属于巍峨耸立的珠穆朗玛峰。它位于喜马拉雅山脉，是中尼两国的天然...,无,0,[unused0]system\n你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一...,[unused0]system\n[unused0]system\n你是一个名字叫做理想同学...,好的，各位观众朋友们，今天我们要聊的话题是这个世界上最高的山。在我们生活的这颗蓝色星球上，巍...,[unused0]system\n[unused0]system\n你是一个名字叫做理想同学...,[unused0] system[unused0] system\n你是一个名字叫做理想同学...,根据参考资料回答问题\n ---\n 下面是给出的实际问题：\n Obse...,尊敬的观众朋友们，大家好，我是康辉。今天我们来探讨一个备受瞩目的地理问题：世界上最高的山是哪...


In [8]:
config = VllmConfig(
    model = 'qwen2_72b',
    url = 'http://172.22.196.240:8000/v1', # 智能云GPT api
    temperature = 1,
    top_p = 0.9,
    max_tokens = 2048, # 最大输出长度
    chunk_num = 1,
    thread_num = 20,
    query_column_name = 'prompts', # llm模型输入列名
    response_column_name = 'qwen2response'
)
call_vllm = CallLLMByVllm(config)

In [9]:
folder = '/workspace/renhuimin/pro_rag/data/distillation_data/v20240724/'
df = pd.read_csv(folder+'手机日志_泛化query.csv')[:5]
oneshot_prompt = '根据参考资料回答问题'
prompt_df = get_prompts_df(df, oneshot_prompt)
responses_tmp = call_vllm.model_request(prompt_df)
responses_tmp


chunk size is 5
chunk_0,range(0,4)


100%|██████████| 5/5 [00:27<00:00,  5.48s/it]

所有线程完成! time cost: 27.47 seconds


,user-query,Thought,API,observation,assistant,system,msg-id,prompts,qwen2response
0,小鹏汽车相比其他汽车品牌有哪些独特的优势,涉及汽车知识问答，查询小鹏汽车有什么优势做参考。该问题请使用总分类模板回复。,"[{'APINAME': 'AUTOSearch', 'CATEGORY': '汽车', '...",[['<该知识来自理想汽车外部数据>\n小鹏汽车质量好吗 有哪些优势 _百度知道 发布时间：...,小鹏汽车的优势主要体现在以下几个方面：\n1. 外观设计：小鹏汽车采用了星际设计元素，贯穿整...,你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一个可靠的智能家庭助手，由理想汽车智...,F8586DE3-03FD-4D8C-99F0-9B6E7843EA1F,根据参考资料回答问题\n ---\n 下面是给出的实际问题：\n Obse...,小鹏汽车相比其他汽车品牌有以下独特优势：\n\n1. **外观设计**：小鹏汽车采用星际设计...
1,小鹏汽车在哪些方面表现出色，使其成为消费者的首选,涉及汽车知识问答，查询小鹏汽车有什么优势做参考。该问题请使用总分类模板回复。,"[{'APINAME': 'AUTOSearch', 'CATEGORY': '汽车', '...",[['<该知识来自理想汽车外部数据>\n小鹏汽车质量好吗 有哪些优势 _百度知道 发布时间：...,小鹏汽车的优势主要体现在以下几个方面：\n1. 外观设计：小鹏汽车采用了星际设计元素，贯穿整...,你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一个可靠的智能家庭助手，由理想汽车智...,F8586DE3-03FD-4D8C-99F0-9B6E7843EA1F,根据参考资料回答问题\n ---\n 下面是给出的实际问题：\n Obse...,小鹏汽车在以下几个方面表现出色，使其成为消费者的首选：\n\n1. **外观设计**：小鹏汽...
2,小鹏汽车的优势主要体现在哪些方面，为什么它在市场上如此受欢迎,涉及汽车知识问答，查询小鹏汽车有什么优势做参考。该问题请使用总分类模板回复。,"[{'APINAME': 'AUTOSearch', 'CATEGORY': '汽车', '...",[['<该知识来自理想汽车外部数据>\n小鹏汽车质量好吗 有哪些优势 _百度知道 发布时间：...,小鹏汽车的优势主要体现在以下几个方面：\n1. 外观设计：小鹏汽车采用了星际设计元素，贯穿整...,你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一个可靠的智能家庭助手，由理想汽车智...,F8586DE3-03FD-4D8C-99F0-9B6E7843EA1F,根据参考资料回答问题\n ---\n 下面是给出的实际问题：\n Obse...,小鹏汽车的优势主要体现在以下几个方面，这也是其在市场上受欢迎的原因：\n\n1. **外观设...
3,理想L9和MEGA之间最大的差异在哪里,涉及汽车知识查询，查询理想L9的简介。和涉及汽车知识查询，查询理想MEGA的简介。,"[{'CATEGORY': 'AUTOSearch', 'QUERY': '汽车', 'TA...",[['理想L9简介 尺寸：全尺寸六座智能电动SUV，定位为家庭智能旗舰SUV。它的车身尺寸为...,理想L9和理想MEGA的最大差异主要体现在车型定位、尺寸、配置和售价上。\n理想L9是一款全...,你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一个可靠的智能家庭助手，由理想汽车智...,4BACAD58-1A3F-495F-8CA9-FC17EEC9BE16,根据参考资料回答问题\n ---\n 下面是给出的实际问题：\n Obse...,理想L9和理想MEGA之间的最大差异在于车型定位、尺寸、性能、配置和售价等方面：\n\n1....
4,你能列举出理想L9和MEGA之间最显著的不同之处吗,涉及汽车知识查询，查询理想L9的简介。和涉及汽车知识查询，查询理想MEGA的简介。,"[{'CATEGORY': 'AUTOSearch', 'QUERY': '汽车', 'TA...",[['理想L9简介 尺寸：全尺寸六座智能电动SUV，定位为家庭智能旗舰SUV。它的车身尺寸为...,理想L9和理想MEGA的最大差异主要体现在车型定位、尺寸、配置和售价上。\n理想L9是一款全...,你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一个可靠的智能家庭助手，由理想汽车智...,4BACAD58-1A3F-495F-8CA9-FC17EEC9BE16,根据参考资料回答问题\n ---\n 下面是给出的实际问题：\n Obse...,理想L9和理想MEGA作为理想汽车旗下不同定位的车型，它们在多个方面存在显著差异，以下是最显...


In [10]:
config = VllmConfig(
    model = 'llama3_70b',
    url = 'http://172.22.196.213:8000/v1', # 智能云GPT api
    temperature = 1,
    top_p = 0.9,
    max_tokens = 2048, # 最大输出长度
    chunk_num = 1,
    thread_num = 20,
    query_column_name = 'prompts', # llm模型输入列名
    response_column_name = 'qwen2response'
)
call_vllm = CallLLMByVllm(config)

In [11]:
folder = '/workspace/renhuimin/pro_rag/data/distillation_data/v20240724/'
df = pd.read_csv(folder+'手机日志_泛化query.csv')[:5]
oneshot_prompt = '根据参考资料回答问题'
prompt_df = get_prompts_df(df, oneshot_prompt)
responses_tmp = call_vllm.model_request(prompt_df)
responses_tmp

chunk size is 5
chunk_0,range(0,4)


100%|██████████| 5/5 [00:23<00:00,  4.64s/it]

所有线程完成! time cost: 23.26 seconds


,user-query,Thought,API,observation,assistant,system,msg-id,prompts,qwen2response
0,小鹏汽车相比其他汽车品牌有哪些独特的优势,涉及汽车知识问答，查询小鹏汽车有什么优势做参考。该问题请使用总分类模板回复。,"[{'APINAME': 'AUTOSearch', 'CATEGORY': '汽车', '...",[['<该知识来自理想汽车外部数据>\n小鹏汽车质量好吗 有哪些优势 _百度知道 发布时间：...,小鹏汽车的优势主要体现在以下几个方面：\n1. 外观设计：小鹏汽车采用了星际设计元素，贯穿整...,你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一个可靠的智能家庭助手，由理想汽车智...,F8586DE3-03FD-4D8C-99F0-9B6E7843EA1F,根据参考资料回答问题\n ---\n 下面是给出的实际问题：\n Obse...,根据观察结果，小鹏汽车相比其他汽车品牌有以下几个独特的优势：\n\n1. 外观方面，小鹏汽车...
1,小鹏汽车在哪些方面表现出色，使其成为消费者的首选,涉及汽车知识问答，查询小鹏汽车有什么优势做参考。该问题请使用总分类模板回复。,"[{'APINAME': 'AUTOSearch', 'CATEGORY': '汽车', '...",[['<该知识来自理想汽车外部数据>\n小鹏汽车质量好吗 有哪些优势 _百度知道 发布时间：...,小鹏汽车的优势主要体现在以下几个方面：\n1. 外观设计：小鹏汽车采用了星际设计元素，贯穿整...,你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一个可靠的智能家庭助手，由理想汽车智...,F8586DE3-03FD-4D8C-99F0-9B6E7843EA1F,根据参考资料回答问题\n ---\n 下面是给出的实际问题：\n Obse...,根据提供的参考资料，根据 Observation 中的信息，小鹏汽车在以下几个方面表现出色，...
2,小鹏汽车的优势主要体现在哪些方面，为什么它在市场上如此受欢迎,涉及汽车知识问答，查询小鹏汽车有什么优势做参考。该问题请使用总分类模板回复。,"[{'APINAME': 'AUTOSearch', 'CATEGORY': '汽车', '...",[['<该知识来自理想汽车外部数据>\n小鹏汽车质量好吗 有哪些优势 _百度知道 发布时间：...,小鹏汽车的优势主要体现在以下几个方面：\n1. 外观设计：小鹏汽车采用了星际设计元素，贯穿整...,你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一个可靠的智能家庭助手，由理想汽车智...,F8586DE3-03FD-4D8C-99F0-9B6E7843EA1F,根据参考资料回答问题\n ---\n 下面是给出的实际问题：\n Obse...,根据提供的参考资料， 小鹏汽车的优势主要体现在以下几个方面：\n\n1. 外观设计：小鹏汽车...
3,理想L9和MEGA之间最大的差异在哪里,涉及汽车知识查询，查询理想L9的简介。和涉及汽车知识查询，查询理想MEGA的简介。,"[{'CATEGORY': 'AUTOSearch', 'QUERY': '汽车', 'TA...",[['理想L9简介 尺寸：全尺寸六座智能电动SUV，定位为家庭智能旗舰SUV。它的车身尺寸为...,理想L9和理想MEGA的最大差异主要体现在车型定位、尺寸、配置和售价上。\n理想L9是一款全...,你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一个可靠的智能家庭助手，由理想汽车智...,4BACAD58-1A3F-495F-8CA9-FC17EEC9BE16,根据参考资料回答问题\n ---\n 下面是给出的实际问题：\n Obse...,根据参考资料，理想L9和MEGA之间最大的差异在于车型类型和尺寸。理想L9是六座智能电动SU...
4,你能列举出理想L9和MEGA之间最显著的不同之处吗,涉及汽车知识查询，查询理想L9的简介。和涉及汽车知识查询，查询理想MEGA的简介。,"[{'CATEGORY': 'AUTOSearch', 'QUERY': '汽车', 'TA...",[['理想L9简介 尺寸：全尺寸六座智能电动SUV，定位为家庭智能旗舰SUV。它的车身尺寸为...,理想L9和理想MEGA的最大差异主要体现在车型定位、尺寸、配置和售价上。\n理想L9是一款全...,你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一个可靠的智能家庭助手，由理想汽车智...,4BACAD58-1A3F-495F-8CA9-FC17EEC9BE16,根据参考资料回答问题\n ---\n 下面是给出的实际问题：\n Obse...,根据参考资料，理想L9和MEGA之间最显著的不同之处是：\n\n1. 车型定位：理想L9是家...
